# Degradation Data Clustering

Titan Hartono (titan.hartono@helmholtz-berlin.de)
Data collected and cleaned from: Paolo Graniero, Hans Koebler

ver 20230816

## 1. Import libraries and load the dataset

### 1.1. Load libraries

In [ ]:
# Install the following packages if they haven't been installed

# pip install "dask[complete]"
# pip install "-U kaleido"
# pip install "kaleido"

In [ ]:
# Import all the packages needed for the notebook to run

# %matplotlib inline
import matplotlib.pyplot as plt
import sys
import os
# import rdkit
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv
from IPython.display import display_html
import seaborn as sns
import json
import pickle5 as pickle
import dask
from PIL import ImageColor

from minisom import MiniSom
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA

In [ ]:
# Set up name to save files
filedirname = '20230816_run_revision_excN2/20230816_sigma_0p5_learningrate_0p1_only3CAT_'

### 1.2. Load dataset in pickle & csv

The data in pickle comes from zenodo: https://doi.org/10.5281/zenodo.8185883.

The data in csv comes from dataset folder.

In [ ]:
# Load PCE_df_grouping, already pre-processed
PCE_df = pd.read_csv('dataset/PCE_df_grouping.csv').drop(['Unnamed: 0'],axis=1)
PCE_df

In [ ]:
# Load the mySeriesDrop that only has MPPTdata
with open('dataset/pkl_complete/20230303_mySeriesDrop.pkl', "rb") as fh:
    mySeriesDrop = pickle.load(fh)

In [ ]:
mySeriesDrop

## 2. Look at the data and do further processing

### 2.1. Plot the PCE_df containing calculated PCE_before (group of PCE) and PCE_delta (change in PCE at 150 hours in comparison to max. PCE)

In [ ]:
# Split the group into 5
n_group = 5

# Print unique values for each group
unique_ceil = PCE_df['PCE_before_ceil_x'].unique()
unique_median = PCE_df['PCE_before_median_x'].unique()
unique_mean = PCE_df['PCE_before_mean_x'].unique()

# Load libraries for plotting
import plotly.express as px
import plotly.graph_objs as go

# Plot boxplot
fig = px.box(PCE_df, x="PCE_before_x", y="PCE_delta")
fig.show()

In [ ]:
# Plot violin plot
fig = px.violin(PCE_df, x="PCE_before_x", y="PCE_delta", 
                box=True, points="all",hover_data=PCE_df.columns)

fig.show()

In [ ]:
import plotly.io as pio
import colorlover as cl
from plotly.colors import n_colors
import matplotlib

fig = go.Figure()

# Label for the groups
a = ['PCE < 10.4%','PCE 10.4-14.2%','PCE 14.2-16.8%','PCE 16.8-19.2%', 'PCE > 19.2%']

print('Unique ceil: ',unique_ceil)
print('Median: ',unique_median)
print('Mean: ',unique_mean)

# Color palette for the figure to make it pretty
colors = n_colors('rgb(8,29,88)', 'rgb(127,205,187)', n_group, colortype='rgb')
colors_box = n_colors('rgb(2,7,22)', 'rgb(30,50,45)', n_group, colortype='rgb')
colors_line = n_colors('rgb(0,5,15)', 'rgb(15,25,23)', n_group, colortype='rgb')

# Plotting the violin and boxplot
for (i,color,color_line) in zip(unique_ceil, colors, colors_line):
    fig.add_trace(go.Violin(x=PCE_df['PCE_before_x'][PCE_df['PCE_before_ceil_x'] == i],
                            y=PCE_df['PCE_delta'][PCE_df['PCE_before_ceil_x'] == i],
                            box_visible=False,
                            fillcolor = color,
                            opacity = 0.4,
                            line = dict(color=color_line),
                            jitter=True,
                            meanline_visible=False))

for (i,color,color_line) in zip(unique_ceil, colors, colors_line):
    fig.add_trace(go.Box(x=PCE_df['PCE_before_x'][PCE_df['PCE_before_ceil_x'] == i],
                            y=PCE_df['PCE_delta'][PCE_df['PCE_before_ceil_x'] == i],
                            marker_color = color,
                            opacity = 0.55,
                            line_color = color_line,
                            fillcolor = color,
                            jitter=True,
                            boxmean=True))

fig.update_layout(xaxis_title="Max. PCE group (%)",
                  yaxis_title="Relative change in max. PCE (after 150 hrs.) (%)",
                  boxgap = 0.85,
                  font_family='Arial',
                  showlegend=False)
    
fig.show()

# Save the figure 
pio.write_image(fig, filedirname+'all_data_changedegradation_4.png',
                width=900, height=600, scale=22)

pio.write_image(fig, filedirname+'all_data_changedegradation_3.png',
                width=700, height=450, scale=25)

pio.write_image(fig, filedirname+'all_data_changedegradation_5.png',
                width=600, height=400, scale=25)



### 2.2. Scaling/ normalization of the MPPT data

There are two types of scaling/ normalization:

1. sklearn.preprocessing.MinMaxScaler -> scaling between min-max of the data (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler)

2. sklearn.preprocessing.MaxAbsScaler -> scaling between 0 and max of the data (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html#sklearn.preprocessing.MaxAbsScaler)

In [ ]:
# Preprocessing: scaling/ normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

# Function to scale, normalize and plot it
def normalize(mySeriesDrop,normalizationMethod):
    '''
    A function to normalize and plot the result
    
    input:
    1. mySeriesDrop (only contains MPPTdata)
    2. normalizationMethod: a string of normalization type, 'MinMaxScaler',
       'MaxAbsScaler'
    
    '''
    mySeriesDrop_norm = mySeriesDrop.copy()
    
    # MinMaxScaler
    if normalizationMethod == 'MinMaxScaler':
        for i in range(len(mySeriesDrop_norm)):
            scaler = MinMaxScaler()
            mySeriesDrop_norm[i] = MinMaxScaler().fit_transform(mySeriesDrop_norm[i])
            mySeriesDrop_norm[i]= mySeriesDrop_norm[i].reshape(len(mySeriesDrop_norm[i]))
    
    # MaxAbsScaler
    elif normalizationMethod == 'MaxAbsScaler':
        for i in range(len(mySeriesDrop_norm)):
            scaler = MaxAbsScaler()
            mySeriesDrop_norm[i] = MaxAbsScaler().fit_transform(mySeriesDrop_norm[i])
            mySeriesDrop_norm[i]= mySeriesDrop_norm[i].reshape(len(mySeriesDrop_norm[i]))
    
    # Plot the first 100 of data
    fig, axs = plt.subplots(10,10,figsize=(30,30), sharex=True, sharey=True)
    for i in range(10):
        for j in range(10):
            if i*10+j+1>len(mySeriesDrop_norm): # pass the others that we can't fill
                continue
            axs[i, j].plot(mySeriesDrop_norm[i*10+j])

    plt.ylim([0,1])
    plt.show()
    
    return mySeriesDrop_norm

In [ ]:
mySeriesDrop_maxAbs = normalize(mySeriesDrop,'MaxAbsScaler')

In [ ]:
# mySeriesDrop_minMax = normalize(mySeriesDrop,'MinMaxScaler')

In [ ]:
# Saving the mySeriesDrop as .pkl file (only has MPPTdata)
# mySeriesDrop_maxAbs.to_pickle('./dataset/pkl_complete/20230116_mySeriesDropNorm.pkl')
mySeriesDrop_maxAbs.to_pickle('./dataset/pkl_complete/20230303_mySeriesDropNorm.pkl')

### 2.3. Smoothing of the MPPT data

Because the data is noisy, we are going to do some 'smoothing' using Savitzky-Golay filter (https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.savgol_filter.html).

In [ ]:
# Load the mySeriesDrop that only has MPPTdata
# with open('dataset/pkl_complete/20230116_mySeriesDropNorm.pkl', "rb") as fh:
#     mySeriesDrop = pickle.load(fh)
with open('dataset/pkl_complete/20230303_mySeriesDropNorm.pkl', "rb") as fh:
    mySeriesDrop = pickle.load(fh)

In [ ]:
mySeriesDrop = mySeriesDrop_maxAbs

In [ ]:
# Plotting the overview of the MPPT data (the first 100 data)

fig, axs = plt.subplots(10,10,figsize=(30,30),sharex=True,sharey=True)

for i in range(10):
    for j in range(10):
        if i*10+j+1>len(mySeriesDrop): # pass the others that we can't fill
            continue
        axs[i, j].plot(mySeriesDrop[i*10+j])
        
plt.ylim([0,1])
plt.show()

We are showing 3 different methods for smoothing:

1. np.convolve (rolling average)

2. scipy.signal.lfilter

3. scipy.signal.savgol (savitzky-golay), which is eventually chosen

In [ ]:
# Convert to rolling average/ np.convolve

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

curve_interest = mySeriesDrop[200]

# Plotting the result
y_smooth_3 = smooth(curve_interest,3)
plt.figure(figsize=(4,2),dpi=300)
plt.plot(curve_interest,'o',alpha=0.05)
plt.plot(smooth(curve_interest,3), 'r-', lw=1)
plt.plot(smooth(curve_interest,50), 'g-', lw=1)

plt.legend(['actual data','convolve:3','convolve:50'])

print(len(curve_interest), len(y_smooth_3))

In [ ]:
# Smoothing using l filter

from scipy.signal import lfilter

n1 = 15  # the larger n is, the smoother curve will be
b1 = [1.0 / n1] * n1
a1 = 1

n2 = 30  # the larger n is, the smoother curve will be
b2 = [1.0 / n2] * n2
a2 = 1

curve_interest = mySeriesDrop[200]

# Plotting with l filter
y_smooth_3 = smooth(curve_interest,3)
plt.figure(figsize=(4,2),dpi=300)
plt.plot(curve_interest,'o',alpha=0.05)
plt.plot(lfilter(b1,a1,curve_interest), 'r-', lw=1)
plt.plot(lfilter(b2,a2,curve_interest), 'g-', lw=1)

plt.legend(['actual data','lfilter n:15','lfilter n:30'])

print(len(curve_interest), len(lfilter(b1,a1,curve_interest)))

In [ ]:
# Using savitzky-golay filter

from scipy.signal import savgol_filter

curve_interest = mySeriesDrop[200] #1195-1224 JSON-lea_1 sample 1 pixel 0

w1 = savgol_filter(curve_interest, 71, 2)
w2 = savgol_filter(curve_interest, 201, 2)

# Plotting the figure
plt.figure(figsize=(4,2),dpi=300)
plt.plot(curve_interest,'o',alpha=0.05)
plt.plot(w1, 'r-', lw=1)
plt.plot(w2, 'g-', lw=1)

print(len(curve_interest), len(w1), len(w2))

plt.legend(['actual data','savgol window:71','savgol window:201'])

Since Savgol with parameter=71 seems to work the best at smoothing, we are going to use that.

**CHECK IF SAVGOL WORKS IN ALL THE ROWS, IF THE FOLLOWING CELL CAN BE RUN WITH NO ERROR, CONTINUE.**

In [ ]:
# Convert to savgol: 71

from scipy.signal import savgol_filter

n = 71
mySeriesDrop_savgol = []

# Calculating savgol series for all the rows
for i in range(len(mySeriesDrop)):
    print('row :',i)
    savgol = savgol_filter(mySeriesDrop[i], n,2)
    mySeriesDrop_savgol.append(savgol)

# Trying to plot after savgol filter
fig, axs = plt.subplots(7,7,figsize=(18,18),sharex=True)
sns.set_style('darkgrid')

for i in range(7):
    for j in range(7):
        if i*7+j+1>len(mySeriesDrop_savgol): # pass the others that we can't fill
            continue
        axs[i, j].plot(mySeriesDrop[i*7+j],'o',color='b',alpha=0.05)#.values)
        axs[i, j].plot(mySeriesDrop_savgol[i*7+j],color='r',lw=2)#.values)

plt.ylim([0,1])
plt.show()

In [ ]:
#### Save numpy array as .npy instead of .pkl
np.save('dataset/pkl_complete/20230303_mySeriesDrop_savgol.npy',mySeriesDrop_savgol)

## 3. SOM/ self-organizing map

Read more about SOM here: https://en.wikipedia.org/wiki/Self-organizing_map.

**input**: clean, pre-processed MPPT data

**process**:

1. cluster them using SOM, explore 3 different parameters combination to see how consistent the clustering results are
2. plot the clusters and distribution
3. split based on the device architecture, plot them
4. look at both clusters and max. PCE group (see if certain clusters correlate with certain max. PCE group more)
5. trendline of relative change -150hrs and the max. PCE group

**output**: 
1. som clusters
2. plots
3. trendline (what is the x-intercept?)

### 3.1. SOM clustering

In [ ]:
# Load the libraries

import math
from minisom import MiniSom
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from PIL import ImageColor

# Load libraries for plotting
import plotly.express as px
import plotly.graph_objs as go

In [ ]:
# Preparing color palettes and opacity

opacity = 0.5

colors=[ImageColor.getcolor(px.colors.qualitative.Pastel1[0],'RGB'),
        ImageColor.getcolor(px.colors.qualitative.Pastel1[1],'RGB'),
        ImageColor.getcolor(px.colors.qualitative.Pastel1[2],'RGB'),
        ImageColor.getcolor(px.colors.qualitative.Pastel1[3],'RGB')]

colors_solid=[ImageColor.getcolor(px.colors.qualitative.Set1[0],'RGB'),
              ImageColor.getcolor(px.colors.qualitative.Set1[1],'RGB'),
              ImageColor.getcolor(px.colors.qualitative.Set1[2],'RGB'),
              ImageColor.getcolor(px.colors.qualitative.Set1[3],'RGB')]

colors_rgba=[]
colors_solid_rgba=[]

for i in range(len(colors)):
    colors_rgba.append('rgba'+str(colors[i])[:-1]+','+str(opacity)+')')
    
for i in range(len(colors_solid)):
    colors_solid_rgba.append('rgba'+str(colors_solid[i])[:-1]+','+str(opacity)+')')

In [ ]:
# Plot with plotly

import plotly.io as pio
import colorlover as cl
from plotly.colors import n_colors
import matplotlib
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Functions to plot the series

# Plot using averaged center
    
def plot_som_series_averaged_center(som_x, som_y, win_map, name):
    
    fig = make_subplots(
    rows=som_x, cols=som_y,
    shared_xaxes=True,
    shared_yaxes=True,
    vertical_spacing=0.1,#0.02,#0.1,
    )
    
    # Colors
    opacity = 0.04

    colors=[ImageColor.getcolor(px.colors.qualitative.Antique[4],'RGB'),
            ImageColor.getcolor(px.colors.qualitative.Antique[9],'RGB'),
            ImageColor.getcolor(px.colors.qualitative.Antique[6],'RGB'),
            ImageColor.getcolor(px.colors.qualitative.Antique[8],'RGB')]

    colors_solid=[ImageColor.getcolor(px.colors.qualitative.Set1[0],'RGB'),
                  ImageColor.getcolor(px.colors.qualitative.Set1[1],'RGB'),
                  ImageColor.getcolor(px.colors.qualitative.Set1[2],'RGB'),
                  ImageColor.getcolor(px.colors.qualitative.Set1[3],'RGB')]

    colors_rgba=[]
    colors_solid_rgba=[]

    for i in range(len(colors)):
        colors_rgba.append('rgba'+str(colors[i])[:-1]+','+str(opacity)+')')

    for i in range(len(colors_solid)):
        colors_solid_rgba.append('rgba'+str(colors_solid[i])[:-1]+','+str(opacity)+')')
    
    # Color count
    color_count = 0
    
    # Time
    time = np.linspace(0,hour_limit, 900, endpoint=True)
    
    # Create the subplots
    for x in range(som_x):
        for y in range(som_y):
            cluster = (x,y)
            cluster_number = x*som_y+y
            if cluster in win_map.keys():

                for series in win_map[cluster]:
                    
                    # Cluster colors
                    if cluster==(0,0):
                        line_color = colors_rgba[0]
                        solid_color = colors_solid_rgba[0]
                    elif cluster==(0,1):
                        line_color = colors_rgba[1]
                        solid_color = colors_solid_rgba[1]
                    elif cluster==(1,0):
                        line_color = colors_rgba[2]
                        solid_color = colors_solid_rgba[2]
                    else:
                        line_color = colors_rgba[3]
                        solid_color = colors_solid_rgba[3]
                        
                    # Plot the traces 
                    fig.add_trace(go.Scatter(x=time, y=series, mode='lines',
                                             name=f"Cluster {cluster_number}",
                                             opacity=0.2,
#                                              line=dict(color='darkgrey'),
                                             line=dict(color=line_color),
                                             showlegend=False),
                                  row=x+1, col=y+1)
                color_count=+1
                
                # Calculate the average
                cluster_mean= np.average(np.vstack(win_map[cluster]),axis=0)
                
                # Plot the average
                fig.add_trace(go.Scatter(x=time, y= cluster_mean, mode='lines',
                                         name=f"Cluster mean {cluster_number}",
                                         line_color='black',
                                         showlegend=False),
                              row=x+1, col=y+1)
            
            # Update the figure
            fig.update_yaxes(range=[-0.1,1.1], row=x+1, col=y+1)
            fig.update_layout(font_family='Arial')

    # Save the figure
    pio.write_image(fig, name+'averagedcenter_'+str(som_x)+'_'+str(som_y)+'.png', 
                    width=1.8*600, height=0.6*600, scale=15) # width=1*600, height=600, scale=15)
    pio.write_image(fig, name+'averagedcenter_big_'+str(som_x)+'_'+str(som_y)+'.png',
                    width=1.8*800, height=0.6*800, scale=15) # width=1*800, height=800, scale=15)
    
    # Showing the figure
    fig.show()
    

# Plot using barycenter 
def plot_som_series_dba_center(som_x, som_y, win_map, name):

    fig = make_subplots(
        rows=som_x, cols=som_y,
        shared_xaxes=True,
        shared_yaxes=True,
        vertical_spacing=0.2,
    )
    
    # Time
    time = np.linspace(0,150, 900, endpoint=True)
    
    # Create the subplots
    for x in range(som_x):
        for y in range(som_y):
            cluster = (x,y)
            cluster_number = x*som_y+y
            if cluster in win_map.keys():
                for series in win_map[cluster]:    
                    
                    # Plot the traces
                    fig.add_trace(go.Scatter(x=time, y=series, mode='lines',
                                             name=f"Cluster {cluster_number}",
                                             line_color='rgba(130,179,196,0.12)',
                                             showlegend=False),
                                  row=x+1, col=y+1)
                
                # Calculate the barycenter average
                cluster_dtw = np.transpose(dtw_barycenter_averaging(np.vstack(win_map[cluster])))
                
                # Plot the barycenter average
                fig.add_trace(go.Scatter(x=time, y= cluster_dtw[0], mode='lines',
                                         name=f"Cluster dtw {cluster_dtw}",
                                         line_color='rgb(57,103,119)',
                                         showlegend=False),
                              row=x+1, col=y+1)
                    
            # Update the figure
            fig.update_yaxes(range=[-0.1,1.1], row=x+1, col=y+1)
            fig.update_layout(font_family='Arial')

    # Save the figure
    pio.write_image(fig, name+'barryaverage.png', width=1*600, height=600, scale=15)
    fig.show()
    

In [ ]:
# Set up name, sigma, learning_rate
sigma = 0.5
learning_rate= 0.1
hour_limit=150

# Set the number of clusters
som_x = 2
som_y = 2
cluster_count= som_x*som_y

In [ ]:
# Reset sns 
sns.reset_orig()

# Calculate the SOM
som = MiniSom(som_x, som_y,len(mySeriesDrop_savgol[0]), sigma=sigma, learning_rate = learning_rate)

som.random_weights_init(mySeriesDrop_savgol)
som.train(mySeriesDrop_savgol, 50000, verbose=True)

# Plot savgol
win_map = som.win_map(mySeriesDrop_savgol)

# Returns the mapping of the winner nodes and inputs
plot_som_series_averaged_center(som_x, som_y, win_map, filedirname)
# plot_som_series_dba_center(som_x, som_y, win_map, filedirname)

In [ ]:
# Sorting the SOM results on win_map keys (to sort based on the shape manually)

# CHANGE SEQUENCE HERE: Turn into single digit keys
win_map[0] = win_map.pop((0,0))
win_map[1] = win_map.pop((0,1))
win_map[2] = win_map.pop((1,0))
win_map[3] = win_map.pop((1,1))

win_map.keys()

In [ ]:
# Now turn back into the mapping (DON'T CHANGE ANYTHING)
win_map[(0,0)] = win_map.pop(0)
win_map[(0,1)] = win_map.pop(1)
win_map[(1,0)] = win_map.pop(2)
win_map[(1,1)] = win_map.pop(3)

# Returns the mapping of the winner nodes and inputs
plot_som_series_averaged_center(som_x, som_y, win_map, filedirname)

Now, after doing the clustering, let's store the data in pandas dataframe.

In [ ]:
# Find out which data row belongs to which cluster
som_shape = (som_x,som_y)
winner_coordinates = np.array([som.winner(x) for x in mySeriesDrop_savgol]).T

# With np.ravel_multi_index we convert the 2-dimensional
# coordinates to a 1-dimensional index
cluster_index_unfixed = np.ravel_multi_index(winner_coordinates, som_shape)

# FIXING the labels sequence to follow the SOM results order
cluster_index = np.empty_like(cluster_index_unfixed)

for i in range(len(cluster_index_unfixed)):
    if(cluster_index_unfixed[i]==0):
        cluster_index[i]=(cluster_index_unfixed[i]+0) # Fix the last value based on sequence 
    elif(cluster_index_unfixed[i]==1):
        cluster_index[i]=(cluster_index_unfixed[i]-0) # Fix the last value based on sequence 
    elif(cluster_index_unfixed[i]==2):
        cluster_index[i]=(cluster_index_unfixed[i]+0) # Fix the last value based on sequence 
    else:
        cluster_index[i]=(cluster_index_unfixed[i]-0) # Fix the last value based on sequence 

# Identify the number of clusters
cluster_c = [len(cluster_index[cluster_index==i]) for i in np.unique(cluster_index)]
cluster_n = ["cluster_"+str(i) for i in np.unique(cluster_index)]

fancy_names_for_labels = [f"{label}" for label in cluster_index]
# result = pd.DataFrame(zip(mySeries['Filename'],mySeries['Pixel'],
#                           mySeries['SampleNumber'],fancy_names_for_labels),
#                       columns=["Series",'Pixel',"SampleNumber","Cluster"]).sort_values(by="Cluster") #.set_index("Series")

result = pd.DataFrame(zip(fancy_names_for_labels),
                      columns=["Cluster"]).sort_values(by="Cluster")

result['PCE_before_x'] = PCE_df['PCE_before_x']
result['PCE_before_ceil_x'] = PCE_df['PCE_before_ceil_x']
result['PCE_before_median_x'] = PCE_df['PCE_before_median_x']
result['PCE_before_mean_x'] = PCE_df['PCE_before_mean_x']
result['PCE_delta'] = PCE_df['PCE_delta']

# Save result on the .csv file
(result.sort_index()).to_csv(filedirname+'clusters.csv')

result.sort_index()

### 3.2. General SOM cluster plots

Now, let's plot some of the results.

In [ ]:
import plotly.express as px

# Plot cluster distribution vertical
fig = px.bar(x=cluster_n, y=cluster_c,labels=dict(x='Clusters',y='Count'))

fig.update_layout(font_family='Arial')
fig.update_traces(marker_color='rgba(57,103,119,0.7)')

# Save the figure
pio.write_image(fig, filedirname+'distribution_v.png',
                width=1*400, height=400, scale=16)
fig.show()

# Plot horizontal distribution 
fig = go.Figure()
fig.add_trace(go.Bar(
    y=cluster_n,
    x=cluster_c,
    orientation='h',
    marker=dict(
        color= 'rgba(57,103,119,0.7)',
    )
))

fig.update_layout(font_family='Arial',
                  xaxis=dict(title='Count'),
                  yaxis=dict(title='Cluster'))

pio.write_image(fig, filedirname+'distribution_h.png', width=1*400, height=1*400, scale=16)
fig.show()

In [ ]:
# Colormap PCE
colormap_PCE = {1.0: "#034e7b",
                2.0: "#0570b0",
                3.0: "#3690c0",
                4.0: "#74a9cf",
                5.0: "#a6bddb"}

# Colormap cluster
colormap_cluster = {"0": px.colors.qualitative.Antique[4],
                    "1": px.colors.qualitative.Antique[9],
                    "2": px.colors.qualitative.Antique[6],
                    "3": px.colors.qualitative.Antique[8]}

# Plot histogram for PCE group distribution
fig = px.histogram(result.sort_values(by=['Cluster','PCE_before_x']),
                   x="Cluster", color='PCE_before_x',
                   color_discrete_map=colormap_PCE, opacity=0.75,
#                    histnorm='percent',
                  )
# Update layout
fig.update_layout(xaxis_title="Cluster",
                  legend_title = 'Max. PCE Group',
                  yaxis_title="Count",
                  font_family='Arial',barmode='group')#,barnorm='fraction')

# Save figure
pio.write_image(fig, filedirname+'group_2.png', width=1*400, height=1*400, scale=16)
fig.show()


# Plot histogram for cluster distribution
fig = px.histogram(result.sort_values(by=['Cluster','PCE_before_x']),
                   x="PCE_before_x", color='Cluster',
                   color_discrete_map=colormap_cluster, opacity=0.7,
#                    histnorm='percent',
                  )

# Update layout
fig.update_layout(xaxis_title="Max. PCE Group (%)",# xaxis=dict(range=[7,25]),
                  yaxis_title="Count", font_family='Arial',barmode='group')#,barnorm='fraction')

# Save figure
pio.write_image(fig, filedirname+'group_3.png', width=1*400, height=1*400, scale=16)
fig.show()

In [ ]:
# Colormap PCE
colormap_PCE = {1.0: "#034e7b",
                2.0: "#0570b0",
                3.0: "#3690c0",
                4.0: "#74a9cf",
                5.0: "#a6bddb"}

# Colormap cluster
colormap_cluster = {"0": px.colors.qualitative.Antique[4],
                    "1": px.colors.qualitative.Antique[9],
                    "2": px.colors.qualitative.Antique[6],
                    "3": px.colors.qualitative.Antique[8]}

# Plot stacked bar based on PCE group
fig = px.histogram(result.sort_values(by=['Cluster','PCE_before_x']),
                   x="Cluster", color='PCE_before_x',
                   color_discrete_map=colormap_PCE, opacity=0.75,
                   histnorm='percent',
                  )

# Update layout
fig.update_layout(xaxis_title="Cluster",
                  legend_title = 'Max. PCE Group',
                  yaxis_title="Count",
                  font_family='Arial')#,barmode='group')#,barnorm='fraction')

# Save figure
pio.write_image(fig, filedirname+'percent_2.png', width=1*400, height=1*400, scale=16)
fig.show()

# Plot stacked bar based on cluster
fig = px.histogram(result.sort_values(by=['Cluster','PCE_before_x']),
                   x="PCE_before_x", color='Cluster',
                   color_discrete_map=colormap_cluster, opacity=0.75,
                   histnorm='percent',
                  )

# Update layout
fig.update_layout(xaxis_title="Max. PCE Group",#xaxis=dict(range=[7,26]),
                  yaxis_title="Count", font_family='Arial',bargap=0.1)#,barmode='group')#,barnorm='fraction')

# Save figure
pio.write_image(fig, filedirname+'percent_3.png', width=1*400, height=1*400, scale=16)
fig.show()

In [ ]:
# Colormap PCE
colormap_PCE = {1.0: "#034e7b",
                2.0: "#0570b0",
                3.0: "#3690c0",
                4.0: "#74a9cf",
                5.0: "#a6bddb"}

# Colormap cluster
colormap_cluster = {"0": px.colors.qualitative.Antique[4],
                    "1": px.colors.qualitative.Antique[9],
                    "2": px.colors.qualitative.Antique[6],
                    "3": px.colors.qualitative.Antique[8]}

# Plot normalized bar based on PCE group
fig = px.histogram(result.sort_values(by=['Cluster','PCE_before_x']),
                   x="Cluster", color='PCE_before_x',
                   color_discrete_map=colormap_PCE, opacity=0.75,
#                    histnorm='percent',
                  )

# Update layout
fig.update_layout(xaxis_title="Cluster",
                  legend_title = 'Max. PCE Group',
                  yaxis_title="Count",
                  font_family='Arial',barnorm='fraction')

# Save figure
pio.write_image(fig, filedirname+'fraction_2.png', width=1*400, height=1*400, scale=16)
fig.show()

# Plot normalized bar based on cluster
fig = px.histogram(result.sort_values(by=['Cluster','PCE_before_x']),
                   x="PCE_before_x", color='Cluster',
                   color_discrete_map=colormap_cluster, opacity=0.75,
#                    histnorm='percent',
                  )

# Update layout
fig.update_layout(xaxis_title="Max. PCE Group",#xaxis=dict(range=[9,26]),
                  yaxis_title="Count", font_family='Arial',barnorm='fraction',
                  bargap=0.1)

# Save figure
pio.write_image(fig, filedirname+'fraction_3.png', width=1*400, height=1*400, scale=16)
fig.show()

### 3.3. Looking at both clusters and max. PCE group

In [ ]:
# Load data for the clusters
cluster_PCE = pd.read_csv(filedirname+'clusters.csv').drop(['Unnamed: 0'],axis=1)
cluster_PCE

In [ ]:
print('Cluster list:')
cluster_PCE['Cluster'].value_counts()

In [ ]:
# Add cluster data to the PCE_df
PCE_df_sorted = ((PCE_df).sort_index()).reset_index(drop=True)
PCE_df_sorted['Cluster'] = cluster_PCE['Cluster']
PCE_df_sorted

In [ ]:
# Separate for different cluster

import plotly.io as pio
import colorlover as cl
from plotly.colors import n_colors
import matplotlib
import random

# Find unique values
unique_x = PCE_df_sorted['PCE_before_x'].unique()

# Colors_scatter
colors_scatter=[ImageColor.getcolor(px.colors.qualitative.Antique[4],'RGB'),
                ImageColor.getcolor(px.colors.qualitative.Antique[9],'RGB'),
                ImageColor.getcolor(px.colors.qualitative.Antique[6],'RGB'),
                ImageColor.getcolor(px.colors.qualitative.Antique[8],'RGB')]

# Plot for each cluster
for cluster in range(4): # Number of clusters: 4
    
    fig = go.Figure()

    colors = n_colors('rgb(8,29,88)', 'rgb(127,205,187)', 6, colortype='rgb')
    colors_box = n_colors('rgb(2,7,22)', 'rgb(30,50,45)', 6, colortype='rgb')
    colors_line = n_colors('rgb(0,5,15)', 'rgb(15,25,23)', 6, colortype='rgb')

    # Plot the violin plot for specific cluster
    for (i,color,color_line) in zip(unique_x, colors, colors_line):
        fig.add_trace(go.Violin(x=(PCE_df['PCE_before_x'][PCE_df['PCE_before_x'] == i])+0,
                                y=PCE_df_sorted['PCE_delta'][(PCE_df_sorted['PCE_before_x'] == i) & (PCE_df_sorted['Cluster']== cluster)],
                                box_visible=False, 
                                fillcolor = color,
                                opacity = 0.4,
                                line = dict(color=color_line),
                                jitter=True,
                                meanline_visible=True
                               )
                     )
    
    # Colors scatter    
    if cluster == 0:
        color_scatter = px.colors.qualitative.Antique[4]#'black'#colors_scatter[0]
    elif cluster == 1:
        color_scatter = px.colors.qualitative.Antique[9]#'blue'#colors_scatter[1]
    elif cluster ==2:
        color_scatter = px.colors.qualitative.Antique[6]#'red'#colors_scatter[2]
    else:
        color_scatter = px.colors.qualitative.Antique[8]#'green'#colors_scatter[3]
    
    # Plot the scattered data points
    for (i) in zip(unique_x):
        x = cluster_PCE['PCE_before_x'][(cluster_PCE['PCE_before_x'] == i) & (cluster_PCE['Cluster'] == cluster)]
        fig.add_trace(go.Scatter(x= x + 0.65*np.random.rand(len(x))-0.325, #-0.5 to center it at 0
                                 y=PCE_df_sorted['PCE_delta'][(PCE_df_sorted['PCE_before_x'] == i) & (PCE_df_sorted['Cluster']== cluster)],
                                 mode='markers',
                                 marker_color=color_scatter,
                                 marker_size=6,
                                 opacity = 0.7,
                                )
                     )

    # Update properties of the figure
    fig.update_traces(marker={'size': 4})
    
    fig.update_layout(xaxis_title="Maximum PCE group (%)",
                      yaxis_title="Relative change in max. PCE (after 150 hrs.) (%)",
                      boxgap = 0.85,
                      font_family='Arial',
                      showlegend=False)
    fig.update_yaxes(range=[-20,120],showgrid=True)
    fig.update_xaxes(showgrid=False,showticklabels=False)
    fig.update_layout(hovermode="y unified")

    fig.show()

    # Save a figure
    pio.write_image(fig, filedirname+'group_violin_cluster_'+str(cluster)+'_2.png',
                    width=425, height=400, scale=20)

### 3.4. Plot the trendline of the mean and interquartile

This will give us some ideas what is the maximum, feasible PCE with 0% relative change in max. PCE after 150 hours.

In [ ]:
import plotly.io as pio
import colorlover as cl
from plotly.colors import n_colors
import matplotlib

##### CALCULATION FOR THE Y-AXIS, THE RELATIVE CHANGE IN MAX.PCE

PCE_df_sorted_upto20 = PCE_df_sorted

# Add the PCE_before_median_x
PCE_df_sorted_upto20['PCE_before_x'] = PCE_df_sorted['PCE_before_x']
PCE_df_sorted_upto20['PCE_before_median_x'] = PCE_df_sorted['PCE_before_median_x']
PCE_df_sorted_upto20['PCE_before_mean_x'] = PCE_df_sorted['PCE_before_median_x']

# Add the PCE_before_mean_x
PCE_df_sorted_upto20['PCE_before_mean_x'].loc[PCE_df_sorted['PCE_before_x'] == 1] = PCE_df_sorted['PCE_before'].loc[PCE_df_sorted['PCE_before_x'] == 1].mean()
PCE_df_sorted_upto20['PCE_before_mean_x'].loc[PCE_df_sorted['PCE_before_x'] == 2] = PCE_df_sorted['PCE_before'].loc[PCE_df_sorted['PCE_before_x'] == 2].mean()
PCE_df_sorted_upto20['PCE_before_mean_x'].loc[PCE_df_sorted['PCE_before_x'] == 3] = PCE_df_sorted['PCE_before'].loc[PCE_df_sorted['PCE_before_x'] == 3].mean()
PCE_df_sorted_upto20['PCE_before_mean_x'].loc[PCE_df_sorted['PCE_before_x'] == 4] = PCE_df_sorted['PCE_before'].loc[PCE_df_sorted['PCE_before_x'] == 4].mean()
PCE_df_sorted_upto20['PCE_before_mean_x'].loc[PCE_df_sorted['PCE_before_x'] == 5] = PCE_df_sorted['PCE_before'].loc[PCE_df_sorted['PCE_before_x'] == 5].mean()

# Extract median
PCE_delta_median = (PCE_df_sorted_upto20.groupby('PCE_before_x')['PCE_delta'].median()).to_frame()
PCE_delta_median = PCE_delta_median.rename_axis('PCE_before_x').reset_index()
PCE_delta_median.rename(columns={'PCE_delta':'PCE_delta_median'},inplace=True)

# Extract mean
PCE_delta_mean = (PCE_df_sorted_upto20.groupby('PCE_before_x')['PCE_delta'].mean()).to_frame()
PCE_delta_mean = PCE_delta_mean.rename_axis('PCE_before_x').reset_index()
PCE_delta_mean.rename(columns={'PCE_delta':'PCE_delta_mean'},inplace=True)

# Extract 25th and 75th percentile
PCE_delta_quartile_1 = PCE_df_sorted_upto20.groupby('PCE_before_x')['PCE_delta'].quantile(0.25).to_frame()
PCE_delta_quartile_3 = PCE_df_sorted_upto20.groupby('PCE_before_x')['PCE_delta'].quantile(0.75).to_frame()

PCE_delta_quartile_1 = PCE_delta_quartile_1.rename_axis('PCE_before_x').reset_index()
PCE_delta_quartile_3 = PCE_delta_quartile_3.rename_axis('PCE_before_x').reset_index()

PCE_delta_quartile_1.rename(columns={'PCE_delta':'PCE_delta_25th'},inplace=True)
PCE_delta_quartile_3.rename(columns={'PCE_delta':'PCE_delta_75th'},inplace=True)


##### CALCULATION FOR THE X-AXIS, THE PCE_before_mean, PCE_before_median, PCE_before_25th, PCE_before_75th
# Extract median x-axis
PCE_before_median = (PCE_df_sorted_upto20.groupby('PCE_before_x')['PCE_before'].median()).to_frame()
PCE_before_median = PCE_before_median.rename_axis('PCE_before_x').reset_index()
PCE_before_median.rename(columns={'PCE_before':'PCE_before_median'},inplace=True)

# Extract mean x-axis
PCE_before_mean = (PCE_df_sorted_upto20.groupby('PCE_before_x')['PCE_before'].mean()).to_frame()
PCE_before_mean = PCE_before_mean.rename_axis('PCE_before_x').reset_index()
PCE_before_mean.rename(columns={'PCE_before':'PCE_before_mean'},inplace=True)

# Extract 25th and 75th percentile
PCE_before_quartile_1 = PCE_df_sorted_upto20.groupby('PCE_before_x')['PCE_before'].quantile(0.25).to_frame()
PCE_before_quartile_3 = PCE_df_sorted_upto20.groupby('PCE_before_x')['PCE_before'].quantile(0.75).to_frame()

PCE_before_quartile_1 = PCE_before_quartile_1.rename_axis('PCE_before_x').reset_index()
PCE_before_quartile_3 = PCE_before_quartile_3.rename_axis('PCE_before_x').reset_index()

PCE_before_quartile_1.rename(columns={'PCE_before':'PCE_before_25th'},inplace=True)
PCE_before_quartile_3.rename(columns={'PCE_before':'PCE_before_75th'},inplace=True)

##### Merge dataframe
PCE_delta_quartiles = pd.concat([PCE_delta_median,
                                 PCE_delta_mean.drop(columns=['PCE_before_x']),
                                 PCE_delta_quartile_1.drop(columns=['PCE_before_x']),
                                 PCE_delta_quartile_3.drop(columns=['PCE_before_x']),
                                 PCE_before_median.drop(columns=['PCE_before_x']),
                                 PCE_before_mean.drop(columns=['PCE_before_x']),
                                 PCE_before_quartile_1.drop(columns=['PCE_before_x']),
                                 PCE_before_quartile_3.drop(columns=['PCE_before_x'])],
                                axis=1)

# Save PCE_delta_quartiles
PCE_delta_quartiles.to_csv(filedirname+'PCE_delta_quartiles_inc24.csv')

PCE_delta_quartiles

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
# from sklearn.datasets import make_regression

# Doing linear regression on MEDIAN
print('For MEDIAN:')
X = (PCE_delta_quartiles['PCE_before_median'].to_numpy()).reshape(-1,1)
y = (PCE_delta_quartiles['PCE_delta_median'].to_numpy()).reshape(-1,1)

# Fitting 
model= LinearRegression().fit(X,y)
model_2 = sm.OLS(y,X).fit()
y_hat = model.predict(X)

# Fitting stretched
X_stretch = np.array([0,35]).reshape(-1,1)
y_hat_stretch = model.predict(X_stretch)

MSE=mean_squared_error(y,y_hat)
MAE=median_absolute_error(y,y_hat)

print('MSE: ',MSE)
print('MAE: ',MAE)
print('Model coefficients: ',model.coef_)
print('y-intercept: ',model.intercept_)
print('x-intercept: ',-model.intercept_/model.coef_)

# Prediction
combined_array = np.column_stack((X,y_hat))
prediction=pd.DataFrame(combined_array,columns=['X','y_hat'])

combined_array_stretch = np.column_stack((X_stretch,y_hat_stretch))
prediction_stretch = pd.DataFrame(combined_array_stretch,
                                  columns=['X','y_hat'])

MAE_np = MAE*np.ones(len(combined_array))
MSE_np = MSE*np.ones(len(combined_array))

# Plot figure
fig = go.Figure()

# Plot the regression line
fig.add_trace(go.Scatter(x=prediction_stretch['X'],y=prediction_stretch['y_hat'],
                         mode='lines', name='Regression',
                         line=dict(dash='dash',color='rgb(116,169,207)')))

# Plot the scatter median line
fig.add_trace(go.Scatter(x=PCE_delta_quartiles['PCE_before_median'],
                         y=PCE_delta_quartiles['PCE_delta_median'],
                         error_y=dict(type='data', symmetric=False,
                                      array=PCE_delta_quartiles['PCE_delta_75th']-PCE_delta_quartiles['PCE_delta_median'],
                                      arrayminus=PCE_delta_quartiles['PCE_delta_median']-PCE_delta_quartiles['PCE_delta_25th']),
                         error_x=dict(type='data', symmetric=False,
                                      array=PCE_delta_quartiles['PCE_before_75th']-PCE_delta_quartiles['PCE_before_median'],
                                      arrayminus=PCE_delta_quartiles['PCE_before_median']-PCE_delta_quartiles['PCE_before_25th']),
                         mode='markers',name='Median',
                         line=dict(color='rgb(5,112,176)')))

# Update figure properties
fig.update_layout(xaxis_title="Maximum PCE group (%)",
                  yaxis_title="Relative change in max. PCE (after 150 hrs.) (%)",
                  font_family='Arial',
                  showlegend=True)


fig.show()

# Save a figure of 300dpi
# pio.write_image(fig, filedirname+'median_PCE_delta_fit_inc24.png',
#                 width=0.7*800, height=0.7*600, scale=12)

pio.write_image(fig, filedirname+'median_PCE_delta_fit_inc24_stretched.png',
                width=0.7*800, height=0.7*600, scale=12)

# Model OLS summary
model_2.summary()

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
# from sklearn.datasets import make_regression

# Doing linear regression on MEDIAN
print('For MEDIAN:')
X = (PCE_delta_quartiles['PCE_before_median'].to_numpy()).reshape(-1,1)
y = (PCE_delta_quartiles['PCE_delta_median'].to_numpy()).reshape(-1,1)

# Fitting 
model= LinearRegression().fit(X,y)
model_2 = sm.OLS(y,X).fit()
y_hat = model.predict(X)

# Fitting stretched
X_stretch = np.array([0,35]).reshape(-1,1)
y_hat_stretch = model.predict(X_stretch)

MSE=mean_squared_error(y,y_hat)
MAE=median_absolute_error(y,y_hat)

print('MSE: ',MSE)
print('MAE: ',MAE)
print('Model coefficients: ',model.coef_)
print('y-intercept: ',model.intercept_)
print('x-intercept: ',-model.intercept_/model.coef_)

# Prediction
combined_array = np.column_stack((X,y_hat))
prediction=pd.DataFrame(combined_array,columns=['X','y_hat'])

combined_array_stretch = np.column_stack((X_stretch,y_hat_stretch))
prediction_stretch = pd.DataFrame(combined_array_stretch,
                                  columns=['X','y_hat'])

MAE_np = MAE*np.ones(len(combined_array))
MSE_np = MSE*np.ones(len(combined_array))

# Plot figure
fig = go.Figure()

# Plot the regression line
fig.add_trace(go.Scatter(x=prediction['X'],y=prediction['y_hat'],
                         mode='lines', name='Regression',
                         line=dict(dash='dash',color='rgb(116,169,207)')))

# Plot the scatter median line
fig.add_trace(go.Scatter(x=PCE_delta_quartiles['PCE_before_median'],
                         y=PCE_delta_quartiles['PCE_delta_median'],
                         error_y=dict(type='data', symmetric=False,
                                      array=PCE_delta_quartiles['PCE_delta_75th']-PCE_delta_quartiles['PCE_delta_median'],
                                      arrayminus=PCE_delta_quartiles['PCE_delta_median']-PCE_delta_quartiles['PCE_delta_25th']),
                         error_x=dict(type='data', symmetric=False,
                                      array=PCE_delta_quartiles['PCE_before_75th']-PCE_delta_quartiles['PCE_before_median'],
                                      arrayminus=PCE_delta_quartiles['PCE_before_median']-PCE_delta_quartiles['PCE_before_25th']),
                         mode='markers',name='Median',
                         line=dict(color='rgb(5,112,176)')))

# Update figure properties
fig.update_layout(xaxis_title="Maximum PCE group (%)",
                  yaxis_title="Relative change in max. PCE (after 150 hrs.) (%)",
                  font_family='Arial',
                  showlegend=True)


fig.show()

# Save a figure of 300dpi
# pio.write_image(fig, filedirname+'median_PCE_delta_fit_inc24.png',
#                 width=0.7*800, height=0.7*600, scale=12)

pio.write_image(fig, filedirname+'median_PCE_delta_fit_inc24.png',
                width=0.7*800, height=0.7*600, scale=12)

# Model OLS summary
model_2.summary()

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
# from sklearn.datasets import make_regression

# Doing linear regression on MEAN
print('For MEAN:')
X = (PCE_delta_quartiles['PCE_before_mean'].to_numpy()).reshape(-1,1)
y = (PCE_delta_quartiles['PCE_delta_mean'].to_numpy()).reshape(-1,1)

# Fitting 
model= LinearRegression().fit(X,y)
model_2 = sm.OLS(y,X).fit()
y_hat = model.predict(X)

# Fitting stretched
X_stretch = np.array([0,35]).reshape(-1,1)
y_hat_stretch = model.predict(X_stretch)

MSE=mean_squared_error(y,y_hat)
MAE=median_absolute_error(y,y_hat)

print('MSE: ',MSE)
print('MAE: ',MAE)
print('Model coefficients: ',model.coef_)
print('y-intercept: ',model.intercept_)
print('x-intercept: ',-model.intercept_/model.coef_)

# Prediction
combined_array = np.column_stack((X,y_hat))
prediction=pd.DataFrame(combined_array,columns=['X','y_hat'])

combined_array_stretch = np.column_stack((X_stretch,y_hat_stretch))
prediction_stretch = pd.DataFrame(combined_array_stretch,
                                  columns=['X','y_hat'])

MAE_np = MAE*np.ones(len(combined_array))
MSE_np = MSE*np.ones(len(combined_array))

# Plot figure
fig = go.Figure()

# Plot the regression line
fig.add_trace(go.Scatter(x=prediction_stretch['X'],y=prediction_stretch['y_hat'],
                         mode='lines', name='Regression',
                         line=dict(dash='dash',color='rgb(116,169,207)')))

# Plot the scatter median line
fig.add_trace(go.Scatter(x=PCE_delta_quartiles['PCE_before_mean'],
                         y=PCE_delta_quartiles['PCE_delta_mean'],
                         error_y=dict(type='data', symmetric=False,
                                      array=PCE_delta_quartiles['PCE_delta_75th']-PCE_delta_quartiles['PCE_delta_mean'],
                                      arrayminus=PCE_delta_quartiles['PCE_delta_mean']-PCE_delta_quartiles['PCE_delta_25th']),
                         error_x=dict(type='data', symmetric=False,
                                      array=PCE_delta_quartiles['PCE_before_75th']-PCE_delta_quartiles['PCE_before_mean'],
                                      arrayminus=PCE_delta_quartiles['PCE_before_mean']-PCE_delta_quartiles['PCE_before_25th']),
                         mode='markers',name='Mean',
                         line=dict(color='rgb(5,112,176)')))

# Update figure properties
fig.update_layout(xaxis_title="Maximum PCE group (%)",
                  yaxis_title="Relative change in max. PCE (after 150 hrs.) (%)",
                  font_family='Arial',
                  showlegend=True)
    
fig.show()

# Save a figure of 300dpi
pio.write_image(fig, filedirname+'mean_PCE_delta_fit_inc24_stretched.png',
                width=0.7*800, height=0.7*600, scale=12)

# Model OLS summary
model_2.summary()

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
# from sklearn.datasets import make_regression

# Doing linear regression on MEAN
print('For MEAN:')
X = (PCE_delta_quartiles['PCE_before_mean'].to_numpy()).reshape(-1,1)
y = (PCE_delta_quartiles['PCE_delta_mean'].to_numpy()).reshape(-1,1)

# Fitting 
model= LinearRegression().fit(X,y)
model_2 = sm.OLS(y,X).fit()
y_hat = model.predict(X)

# Fitting stretched
X_stretch = np.array([0,35]).reshape(-1,1)
y_hat_stretch = model.predict(X_stretch)

MSE=mean_squared_error(y,y_hat)
MAE=median_absolute_error(y,y_hat)

print('MSE: ',MSE)
print('MAE: ',MAE)
print('Model coefficients: ',model.coef_)
print('y-intercept: ',model.intercept_)
print('x-intercept: ',-model.intercept_/model.coef_)

# Prediction
combined_array = np.column_stack((X,y_hat))
prediction=pd.DataFrame(combined_array,columns=['X','y_hat'])

combined_array_stretch = np.column_stack((X_stretch,y_hat_stretch))
prediction_stretch = pd.DataFrame(combined_array_stretch,
                                  columns=['X','y_hat'])

MAE_np = MAE*np.ones(len(combined_array))
MSE_np = MSE*np.ones(len(combined_array))

# Plot figure
fig = go.Figure()

# Plot the regression line
fig.add_trace(go.Scatter(x=prediction['X'],y=prediction['y_hat'],
                         mode='lines', name='Regression',
                         line=dict(dash='dash',color='rgb(116,169,207)')))

# Plot the scatter median line
fig.add_trace(go.Scatter(x=PCE_delta_quartiles['PCE_before_mean'],
                         y=PCE_delta_quartiles['PCE_delta_mean'],
                         error_y=dict(type='data', symmetric=False,
                                      array=PCE_delta_quartiles['PCE_delta_75th']-PCE_delta_quartiles['PCE_delta_mean'],
                                      arrayminus=PCE_delta_quartiles['PCE_delta_mean']-PCE_delta_quartiles['PCE_delta_25th']),
                         error_x=dict(type='data', symmetric=False,
                                      array=PCE_delta_quartiles['PCE_before_75th']-PCE_delta_quartiles['PCE_before_mean'],
                                      arrayminus=PCE_delta_quartiles['PCE_before_mean']-PCE_delta_quartiles['PCE_before_25th']),
                         mode='markers',name='Mean',
                         line=dict(color='rgb(5,112,176)')))

# Update figure properties
fig.update_layout(xaxis_title="Maximum PCE group (%)",
                  yaxis_title="Relative change in max. PCE (after 150 hrs.) (%)",
                  font_family='Arial',
                  showlegend=True)
    
fig.show()

# Save a figure of 300dpi
pio.write_image(fig, filedirname+'mean_PCE_delta_fit_inc24.png',
                width=0.7*800, height=0.7*600, scale=12)

# Model OLS summary
model_2.summary()

## 4. Time series k-means clustering

Read more about k-means clustering here: https://en.wikipedia.org/wiki/K-means_clustering.

**input**: clean, pre-processed mySeriesDrop_savgol from above

**process**:

1. do k-means clustering with the same number of clusters as som
2. visualization using PCA: k-means, affinity propagation, and dbscan (this one is too long)
3. visualization using t-SNE: k-means, affinity propagation
4. plot the cluster distribution
5. plot elbow method and silhouette method for optimum number of clusters

**output**: 
1. optimum number for clustering
2. 2d map of the data points, and their clusters

### 4.1. Direct k-means clustering

In [ ]:
# Load mySeriesDrop_savgol, uncomment to check:
# mySeriesDrop_savgol

# Load mySeriesDrop_savgol
mySeriesDrop_savgol=np.load('dataset/pkl_complete/20230303_mySeriesDrop_savgol.npy')

Beware, the following cell takes a really long time!

In [ ]:
import math

# Using the same number of clusters as the SOM
som_x = 2
som_y = 2
cluster_count= som_x*som_y

# K-means clustering
km = TimeSeriesKMeans(n_clusters=cluster_count, metric="dtw")
labels = km.fit_predict(mySeriesDrop_savgol)

In [ ]:
# Saving models as pickle
km.to_pickle(filedirname+'TimeSeriesKMeans_4clusters_2.pkl')

# Save numpy array as .npy instead of .pkl
np.save(filedirname+'TimeSeriesKMeans_labels_dtw_4clusters_2.npy',labels)

In [ ]:
# Set up name to save files

# filedirname = '20230116_run/sigma_0p5_learningrate_0p1/20230116_sigma_0p5_learningrate_0p1_'

# If not, let's load labels and km
with open(filedirname+'TimeSeriesKMeans_4clusters_2.pkl', "rb") as fh:
    km = pickle.load(fh)
labels=np.load(filedirname+'TimeSeriesKMeans_labels_dtw_4clusters_2.npy')

In [ ]:
# Fixing the labels sequence to follow the SOM results order

labels_fixed = np.empty_like(labels)
for i in range(len(labels)):
    if(labels[i]==0):
        labels_fixed[i]=(labels[i]+1) # Fix based on the sequence
    elif(labels[i]==1):
        labels_fixed[i]=(labels[i]+1) # Fix based on the sequence
    elif(labels[i]==2):
        labels_fixed[i]=(labels[i]+1) # Fix based on the sequence
    else:
        labels_fixed[i]=(labels[i]-3) # Fix based on the sequence

In [ ]:
from plotly.subplots import make_subplots
import plotly.io as pio
import colorlover as cl
from plotly.colors import n_colors
import plotly.graph_objects as go
import plotly.express as px

# Now, let's plot the results (based on the savgol/ smoothed )
plot_count = som_x

# Time
time = np.linspace(0,150, 900, endpoint=True)

# Plot figure
fig = make_subplots(
    rows=som_x, cols=som_y,
    shared_xaxes=True,
    shared_yaxes=True,
    vertical_spacing=0.1,
    )

row_i = 0
column_j = 0

# Colors
opacity = 0.04

colors=[ImageColor.getcolor(px.colors.qualitative.Antique[4],'RGB'),
        ImageColor.getcolor(px.colors.qualitative.Antique[9],'RGB'),
        ImageColor.getcolor(px.colors.qualitative.Antique[6],'RGB'),
        ImageColor.getcolor(px.colors.qualitative.Antique[8],'RGB')]

colors_solid=[ImageColor.getcolor(px.colors.qualitative.Set1[0],'RGB'),
              ImageColor.getcolor(px.colors.qualitative.Set1[1],'RGB'),
              ImageColor.getcolor(px.colors.qualitative.Set1[2],'RGB'),
              ImageColor.getcolor(px.colors.qualitative.Set1[3],'RGB')]

colors_rgba=[]
colors_solid_rgba=[]


for i in range(len(colors)):
    colors_rgba.append('rgba'+str(colors[i])[:-1]+','+str(opacity)+')')

for i in range(len(colors_solid)):
    colors_solid_rgba.append('rgba'+str(colors_solid[i])[:-1]+','+str(opacity)+')')

# For each label there is, plot every series with that label
for label in set(labels_fixed):
    cluster = []
    
    # Plot for the labels
    for i in range(len(labels_fixed)):
        if(labels_fixed[i]==label):
            
            # Cluster colors
            if label==0:
                line_color = colors_rgba[0]
                solid_color = colors_solid_rgba[0]
            elif label==1:
                line_color = colors_rgba[1]
                solid_color = colors_solid_rgba[1]
            elif label==2:
                line_color = colors_rgba[2]
                solid_color = colors_solid_rgba[2]
            else:
                line_color = colors_rgba[3]
                solid_color = colors_solid_rgba[3]
            
            # Add trace
            fig.add_trace(go.Scatter(x=time, y=mySeriesDrop_savgol[i],
                                     mode='lines',
                                     name=f"Cluster {label}",
                                     line_color=line_color,#'rgba(130,179,196,0.12)',
                                     showlegend=False),
                          row=row_i+1, col=column_j+1)
            cluster.append(mySeriesDrop_savgol[i]) # Append the series to take the average
    
    # Plot the average within the cluster
    if len(cluster) > 0:
        fig.add_trace(go.Scatter(x=time,y=np.average(np.vstack(cluster),axis=0),
                                 mode='lines',
                                 name=f'Cluster {label}',
                                 line_color='black',#'rgb(57,103,119)',
                                 showlegend=False),
                      row=row_i+1, col=column_j+1)
    
    # Go to the next row, column
    column_j+=1
    if column_j%plot_count == 0:
        row_i+=1
        column_j=0

# Update the figure
fig.update_yaxes(range=[-0.1,1.1])#, row=x+1, col=y+1)
fig.update_layout(font_family='Arial')

# Save the figure
pio.write_image(fig, filedirname+'TimeSeriesKMeans_4clusters_dtw_2.png', width=1*600, height=600, scale=15)
      
fig.show()

In [ ]:
type(labels_fixed)

In [ ]:
# Count for each cluster
cluster_c = [len(labels_fixed[labels_fixed==i]) for i in range(cluster_count)]
cluster_n = ["cluster_"+str(i) for i in range(cluster_count)]

import plotly.express as px

# Plot the bar plot of cluster distribution
fig = px.bar(x=cluster_n, y=cluster_c,labels=dict(x='Clusters',y='Count'))

fig.update_layout(font_family='Arial')
fig.update_traces(marker_color='rgba(57,103,119,0.7)')

# Save the figure
pio.write_image(fig, filedirname+'distribution_TimeSeriesKMeans_4clusters_v_2.png',
                width=1*400, height=400, scale=16)
fig.show()

In [ ]:
import plotly.express as px

# Plot horizontal distribution
fig = go.Figure()
fig.add_trace(go.Bar(
    y=cluster_n,
    x=cluster_c,
    orientation='h',
    marker=dict(
        color= 'rgba(57,103,119,0.7)',
    )
))

fig.update_layout(font_family='Arial',
                  xaxis=dict(title='Count'),
                  yaxis=dict(title='Cluster'))

# Save the figure
pio.write_image(fig, filedirname+'distribution_TimeSeriesKMeans_4clusters_h_2.png',
                width=1*400, height=1*400, scale=16)

fig.show()

In [ ]:
# Save the clustering results in one dataframe
fancy_names_for_labels = [f"{label}" for label in labels_fixed]
result = pd.DataFrame(zip(mySeries['Filename'],mySeries['Pixel'],mySeries['SampleNumber'],fancy_names_for_labels),
                      columns=["Series",'Pixel',"SampleNumber","Cluster"]).sort_values(by="Cluster")#.set_index("Series")

result['PCE_before_x'] = PCE_df['PCE_before_x']
result['PCE_before_ceil_x'] = PCE_df['PCE_before_ceil_x']
result['PCE_before_median_x'] = PCE_df['PCE_before_median_x']
result['PCE_before_mean_x'] = PCE_df['PCE_before_mean_x']

# Save the results as csv
(result.sort_index()).to_csv(filedirname+'TimeSeriesKMeans_DTW_4clusters_newdata_2.csv')
(result.sort_index())

In [ ]:
# Plot the distribution

colormap_cluster = {"0": "#045a8d",
                    "1": "#2b8cbe",
                    "2": "#74a9cf",
                    "3": "#bdc9e1"}

# Colormap cluster
colormap_cluster = {"0": px.colors.qualitative.Antique[4],
                    "1": px.colors.qualitative.Antique[9],
                    "2": px.colors.qualitative.Antique[6],
                    "3": px.colors.qualitative.Antique[8]}

# Plot the figure
fig = px.histogram(result.sort_values(by=['Cluster','PCE_before_ceil_x']),
                   x="PCE_before_x", color='Cluster',
                   color_discrete_map=colormap_cluster, opacity=0.75,
                  )

# fig.update_traces(xbins_size=2)

fig.update_layout(xaxis_title="Max. PCE Group",#xaxis=dict(range=[9,26]),
                  yaxis_title="Count", font_family='Arial',barnorm='fraction',
                  bargap=0.1)

pio.write_image(fig, filedirname+'TimeSeriesKMeans_4clusters_fraction_3_2.png',
                width=1*400, height=1*400, scale=16)

fig.show()

## 5. Side notes/ figures

### 5.1. The number of each cluster for each max. PCE group

In [ ]:
# Load the data of interest
cluster_PCE_int = pd.read_csv(filedirname+'clusters.csv').drop(['Unnamed: 0'],axis=1)
PCE_df_group_int = pd.read_csv('dataset/PCE_df_grouping.csv')
PCE_df_group_int_sort = (PCE_df_group_int.sort_values('Unnamed: 0')).reset_index()

cluster_PCE_int['PCE_delta']=PCE_df_group_int_sort['PCE_delta']
PCE_df_group_int_sort

In [ ]:
# Group of interest calculation

for i in range(4): # Clusters
    for j in range(5): # PCE_before_x
        count = cluster_PCE_int.loc[(cluster_PCE_int['Cluster']==i)&(cluster_PCE_int['PCE_before_x']==j+1)]
        print('For cluster: ',i, ' and PCE_before_x: ',j+1,
              '# data points:', count.shape[0])

### 5.2. Histogram cluster

In [ ]:
# Grouping the data
cluster_1 = cluster_PCE_int['PCE_delta'].loc[(cluster_PCE_int['Cluster']==0)]
cluster_2 = cluster_PCE_int['PCE_delta'].loc[(cluster_PCE_int['Cluster']==1)]
cluster_3 = cluster_PCE_int['PCE_delta'].loc[(cluster_PCE_int['Cluster']==2)]
cluster_4 = cluster_PCE_int['PCE_delta'].loc[(cluster_PCE_int['Cluster']==3)]

# Colors, labels, and data for plotting
hist_data = [cluster_1, cluster_2, cluster_3, cluster_4]
group_labels = ['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4']
colors = [px.colors.qualitative.Antique[4],
          px.colors.qualitative.Antique[9],
          px.colors.qualitative.Antique[6],
          px.colors.qualitative.Antique[8]]

In [ ]:
import plotly.figure_factory as ff

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=2.5,
                         colors=colors)

fig.update_layout(font_family='Arial')
pio.write_image(fig, filedirname+'rug_dist_PCE_delta.png',
                width=1*900, height=1*400, scale=12)
pio.write_image(fig, filedirname+'rug_dist_PCE_delta_2.png',
                width=1*600, height=1*400, scale=12)
fig.show()

# Create distplot inset (the top)
fig = ff.create_distplot(hist_data, group_labels, bin_size=2.5,
                         colors=colors)

fig.update_layout(font_family='Arial')
fig.update_xaxes(range=[-2,90])
fig.update_yaxes(range=[-0.005,0.07])
pio.write_image(fig, filedirname+'rug_dist_PCE_delta_inset_top.png',
                width=1*900, height=1*400, scale=12)
pio.write_image(fig, filedirname+'rug_dist_PCE_delta_inset_top_2.png',
                width=1*600, height=1*400, scale=12)
fig.show()

# Create distplot inset (the bottom)
fig = ff.create_distplot(hist_data, group_labels, bin_size=2.5,
                         colors=colors)

fig.update_layout(font_family='Arial')
fig.update_xaxes(range=[-2,90])
pio.write_image(fig, filedirname+'rug_dist_PCE_delta_inset_bottom.png',
                width=1*900, height=1*400, scale=12)
pio.write_image(fig, filedirname+'rug_dist_PCE_delta_inset_bottom_2.png',
                width=1*600, height=1*400, scale=12)
fig.show()

### 5.3. Quantization error SOM


In [ ]:
# These values are generated by trying out different combinations for
# som_x and som_y in the main part of the code
x = np.array((2,3,4,5,6,7,8,9,10))

y = np.array((3.4653711937400833,
              2.7734776665323935,
              (2.4188140391965622+2.4188140345317097)/2,
              1.97200974884117,
              (1.8233955273321616+1.823395449115016)/2,
              1.791192306527944,
              (1.6143132145605121+1.6141542228347283)/2,
              (1.5764475628465437+1.5582839343215373)/2,
              (1.4706708588067163+1.4986430155342734)/2))

In [ ]:
# Plot quantization error
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='lines+markers',
    line=dict(color='rgb(5,112,176)'),
))

fig.update_layout(font_family='Arial',
                  xaxis=dict(title='Number of clusters'),
                  yaxis=dict(title='Quantization error'))

# Save the figure
pio.write_image(fig, filedirname+'som_quantizationError_result_cluster_2_10_mySeriesDrop_savgol.png',
                width=1*300, height=1*300, scale=16)

fig.show()